# Caso de estudio: función Q

El objetivo de este notebook es repasar algunas operaciones con arrays, especialmente `np.meshgrid`. Para ello, trabajaremos sobre el siguiente problema:

Dados unos datos $D = \{d_1, d_2, \ldots d_n\} $ define función

$$ Q(x,D) = \sum_{k=1}^n (x-d_k)^2 $$

que es la suma de diferencias al cuadrado entre el argumento $x$ y los datos $D$, y calcula el valor de $x$ donde se alcanza el mínimo de $Q$.

### Exploración del problema

En primer lugar importamos los paquetes habituales:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(5,3))

Para hacer gráficas con `plt.plot` definimos un array de valores que recorren el intervalo de interés de la variable.

In [ ]:
X = np.linspace(-4,18,100)

Supongamos que los datos son $D = \{3,7,8\}$.

Para cada dato concreto el término correspondiente del sumatorio es una parábola:

In [ ]:
d = 3

plt.plot(X, (X-d)**2 );

Observa que la expresión `(X-3)**2` es un array, se calculan automáticamente todos los valores correspondientes al vector `X`.

Podemos dibujar varios términos del sumatorio, que son diferentes parábolas centradas en cada dato:

In [ ]:
plt.plot(X, (X-3)**2 )
plt.plot(X, (X-7)**2 )
plt.plot(X, (X-8)**2 );

La suma de parábolas es también una parábola:

In [ ]:
plt.plot(X, (X-3)**2 + (X-7)**2 + (X-8)**2 );

Obtenemos el mismo resultado con un bucle implícito:

In [ ]:
plt.plot(X, sum([ (X-d)**2 for d in [3,7,8]]) );

### Definición de la función 

Lo expresamos en forma de función. Tiene dos argumentos, uno de ellos es el conjunto de datos, claramente un array, y el otro el número real $x$ donde se evalúa la función. También queremos que $x$ pueda ser un array, para poder evaluar directamente la función en un conjunto de elementos del dominio como en las gráficas anteriores.

Son dos argumentos de tipo array, pero son dos variables independientes, los elementos no van emparejados, sino que se combinan todos con todos, por lo que lo más práctico es utilizar `meshgrid` (explicado en las diapositivas de clase):

In [ ]:
def Q(xs, datos):
    x,d = np.meshgrid(xs,datos)
    return np.sum( (x-d)**2, axis=0 )

In [ ]:
D = [1,3,10,12,13]

plt.plot( X, Q(X, D) );

In [ ]:
Q(2, D)

In [ ]:
Q( np.array( [1,2,3]), D )

### Minimización (método aproximado)

La segunda parte del problema consiste en obtener $\DeclareMathOperator*{\argminA}{arg\,min} \argminA_x Q(x,D)$.

Una primera forma aproximada de hacerlo es aplicar operaciones básicas de arrays a las secuencias de valores `X` y`Q(x,D)` que hemos usado para dibujar. La posición donde se alcanza el mínimo es:

In [ ]:
k = np.argmin( Q(X,D) )
k

El valor de $x$ correspondiente a esa posición es:

In [ ]:
X[k]

Con lo que quedaría resuelto el problema.

Es inmediato calcular también el valor de la función en el mínimo:

In [ ]:
Q( X[k], D )

O también

In [ ]:
Q(X, D)[k]

Las dos expresiones anteriores son equivalentes. Explícalo.

Este método no es muy bueno, ya que solo se tienen en cuenta los elementos discretos del array `X` generado por `np.linspace`. Para encontrar así el mínimo con precisión se necesita un muestreo muy denso del dominio, lo que aumenta el tiempo de cómputo y el espacio de almacenamiento. Esto puede llegar a ser prohibitivo en problemas de varias variables.

### Minimización (método preciso)

Es mejor utilizar `minimize`:

In [ ]:
from scipy.optimize import minimize

Solo necesitamos indicar la función a minimizar, el punto de partida y los posible argumentos adicionales:

In [ ]:
#              función a minimizar (su primer argumento)                 
#              |  punto de partida
#              |  |       argumentos extra de la función a minimizar
#              |  |       |
#              V  V       V 
sol = minimize(Q, 0, args=D)
sol

In [ ]:
sol['x']

Sabemos que la media de los datos es el valor que produce menor error cuadrático. Por tanto la solución exacta es:

In [ ]:
np.mean(D)

Los mensajes impresos nos informan de que el algoritmo de minimización (una variante del método de Newton) ha realizado solo 12 evaluaciones de la función (`nfev`) y consigue un error relativo menor que $10^{-8}$.

### Simplificación

Manipulando la suma de cuadrados como se explicó en clase podemos expresar la función de forma mucho más simple, donde se ve directamente que el mínimo se obtiene en el valor medio de los datos:

In [ ]:
def Qm(x, datos):
    mx  = np.mean(datos)
    var = np.var(datos)
    n = len(datos)
    return n*(x-mx)**2  + n*var

In [ ]:
Qm( 3, [3,4,5,6])

In [ ]:
Qm( np.array([3,8,10]) , [123,44,57,60])